## Reference
- [TensorFlow Example - convolution network](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3%20-%20Neural%20Networks/convolutional_network.ipynb)

In [1]:
import tensorflow as tf
import numpy as np
import input_data

mnist = input_data.read_data_sets("./../data/MNIST/", one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/Dongjun/DLselfProjects/4.Convolution_Network/input_data.py:41: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting ./../data/MNIST/train-images-idx3-ubyte.gz
Extracting ./../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./../data/MNIST/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.01
num_iters = 1000
batch_size = 50

geometry = [28, 28]
classes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
num_classes = len(classes)
dropout = 0.5

In [3]:
# Tensor Flow Graph Input
X = tf.placeholder(tf.float32, [None, geometry[0]*geometry[1]])
y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Weight & bias
# 5x5 conv, 1 input, 32 outputs
Wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

# 5x5 conv, 32 input, 64 outputs
Wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

# Fully connected (Standard 2-layer MLP), 7*7*64 input, 1024 
Wf1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bf1 = tf.Variable(tf.random_normal([1024]))

Wf2 = tf.Variable(tf.random_normal([1024, num_classes]))
bf2 = tf.Variable(tf.random_normal([num_classes]))

In [4]:
# Convolution Network

# Reshape input picture
input_X = tf.reshape(X, shape=[-1, 28, 28, 1])

# Stage 1 : Convolution -> ReLU -> Max Pooling -> Dropout
conv1 = tf.nn.conv2d(input_X, Wc1, strides = [1, 1, 1, 1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, bc1))
conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')
conv1 = tf.nn.dropout(conv1, dropout)

# Stage 2 : Convolution -> ReLU -> Max Pooling -> Dropout
conv2 = tf.nn.conv2d(conv1, Wc2, strides = [1, 1, 1, 1], padding='SAME')
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, bc2))
conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')
conv2 = tf.nn.dropout(conv2, dropout)

# Stage 3 : Fully connected : Linear -> ReLU -> Dropout
fc1 = tf.reshape(conv2, [-1, Wf1.get_shape().as_list()[0]])
fc1 = tf.nn.relu(tf.add(tf.matmul(fc1, Wf1), bf1))
fc1 = tf.nn.dropout(fc1, dropout)

fc2 = tf.add(tf.matmul(fc1, Wf2), bf2)

In [5]:
# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc2, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
init = tf.initialize_all_variables()

# Launch the Graph
with tf.Session() as sess:
    sess.run(init)
    
    # Train
    for epoch in range(1, num_iters+1):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training data
        sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, keep_prob: dropout})
        
        if epoch & 50 == 0:
            loss = sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, keep_prob: 1.})
            print("Epoch : ", epoch, " loss=" , loss)
    
    print("Optimization Finishied")
    
    # Test
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                             y: mnist.test.labels, 
                                                             keep_prob: 1.}) )

Epoch :  1  loss= 185346.0
Epoch :  4  loss= 164579.0
Epoch :  5  loss= 175150.0
Epoch :  8  loss= 133817.0
Epoch :  9  loss= 84569.4
Epoch :  12  loss= 98107.3
Epoch :  13  loss= 105425.0
Epoch :  64  loss= 10524.8
Epoch :  65  loss= 19349.1
Epoch :  68  loss= 13508.3
Epoch :  69  loss= 15865.4
Epoch :  72  loss= 11148.6
Epoch :  73  loss= 15532.0
Epoch :  76  loss= 11028.9
Epoch :  77  loss= 8653.56
Epoch :  128  loss= 5556.6
Epoch :  129  loss= 4090.4
Epoch :  132  loss= 6655.88
Epoch :  133  loss= 5203.44
Epoch :  136  loss= 5853.5
Epoch :  137  loss= 5661.03
Epoch :  140  loss= 5765.39
Epoch :  141  loss= 4887.99
Epoch :  192  loss= 4828.13
Epoch :  193  loss= 4551.56
Epoch :  196  loss= 3747.08
Epoch :  197  loss= 6134.83
Epoch :  200  loss= 3721.02
Epoch :  201  loss= 2605.82
Epoch :  204  loss= 4083.76
Epoch :  205  loss= 3143.31
Epoch :  256  loss= 3494.73
Epoch :  257  loss= 2883.45
Epoch :  260  loss= 2413.18
Epoch :  261  loss= 2585.55
Epoch :  264  loss= 2718.13
Epoch :  2